In [63]:
import pandas as pd
import re
from langchain.embeddings import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

c:\Users\param\Core\Code\SHL AI Intern Hiring 2025\RAG\shl_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
df = pd.read_csv('../assessment_details.csv')

In [48]:
df.head()

,name,url,description,job_levels,languages,duration,test_type,remote_testing,adaptive_irt
0,Account Manager Solution,https://www.shl.com/solutions/products/product...,The Account Manager solution is an assessment ...,Mid-Professional,English (USA),49.0,"C,P,A,B",True,True
1,Administrative Professional - Short Form,https://www.shl.com/solutions/products/product...,The Administrative Professional solution is fo...,Entry-Level,English (USA),36.0,"A,K,P",True,True
2,Agency Manager Solution,https://www.shl.com/solutions/products/product...,The Agency Manager solution is for mid-level s...,"Front Line Manager, Manager, Supervisor",English (USA),51.0,"A,B,P,S",True,True
3,Apprentice + 8.0 Job Focused Assessment,https://www.shl.com/solutions/products/product...,The Apprentice + 8.0 Job-Focused Assessment is...,"General Population, Graduate, Entry-Level","English International, German",30.0,"B,P",True,False
4,Apprentice 8.0 Job Focused Assessment,https://www.shl.com/solutions/products/product...,The Apprentice 8.0 Job-Focused Assessment is a...,"Entry-Level, General Population, Graduate","English International, German, French",20.0,"B,P",True,False


In [49]:
# Check for null values in each column
null_values = df.isnull().sum()
print("Null values per column:")
print(null_values)
print("\n")

# Check data types
print("Data types:")
print(df.dtypes)
print("\n")

# Check for consistency issues
print("Column value counts:")
print(f"remote_testing: {df['remote_testing'].value_counts()}")
print(f"adaptive_irt: {df['adaptive_irt'].value_counts()}")
print("\n")

# Check test_type format consistency
print("Sample of test_type values:")
print(df['test_type'].head(10))
print("\n")

# Check duration statistics
print("Duration statistics:")
print(df['duration'].describe())
print("\n")

# Check if any required columns are missing
expected_cols = ['name', 'url', 'description', 'job_levels', 'languages', 
                 'duration', 'test_type', 'remote_testing', 'adaptive_irt']
missing_cols = [col for col in expected_cols if col not in df.columns]
if missing_cols:
    print(f"Missing columns: {missing_cols}")
else:
    print("All expected columns are present")

Null values per column:
name               0
url                0
description        0
job_levels        16
languages         36
duration          84
test_type          0
remote_testing     0
adaptive_irt       0
dtype: int64


Data types:
name               object
url                object
description        object
job_levels         object
languages          object
duration          float64
test_type          object
remote_testing       bool
adaptive_irt         bool
dtype: object


Column value counts:
remote_testing: remote_testing
True    398
Name: count, dtype: int64
adaptive_irt: adaptive_irt
False    351
True      47
Name: count, dtype: int64


Sample of test_type values:
0      C,P,A,B
1        A,K,P
2      A,B,P,S
3          B,P
4          B,P
5      A,B,K,P
6        A,B,P
7      A,B,P,S
8      B,P,S,A
9    P,S,K,B,A
Name: test_type, dtype: object


Duration statistics:
count    314.000000
mean      14.917197
std       11.211280
min        0.000000
25%        8.000000
50%    

In [50]:
# Get all rows with at least one null value
null_rows = df[df.isnull().any(axis=1)]

# Display the number of rows with null values
print(f"Number of rows with null values: {len(null_rows)}")

# Display these rows
null_rows

Number of rows with null values: 87


,name,url,description,job_levels,languages,duration,test_type,remote_testing,adaptive_irt
12,Global Skills Development Report,https://www.shl.com/solutions/products/product...,This report is designed to be given to individ...,"Director, Entry-Level, Executive, General Popu...",NaN,NaN,"A,E,B,C,D,P",True,False
24,Executive Scenarios Profile Report,https://www.shl.com/solutions/products/product...,Executive Scenarios is a unique test of Manage...,"Director, Executive, Manager",NaN,NaN,B,True,False
42,Global Skills Assessment,https://www.shl.com/solutions/products/product...,The Global Skills Assessment (GSA) is an asses...,NaN,"Indonesian, Italian, Swedish, Thai, Portuguese...",16.0,"C,K",True,False
43,Graduate Scenarios,https://www.shl.com/solutions/products/product...,Graduate Scenarios is a unique test of Manager...,"Manager, Mid-Professional, Professional Indivi...",English International,NaN,B,True,False
44,Graduate Scenarios Narrative Report,https://www.shl.com/solutions/products/product...,Graduate Scenarios is a unique test of Manager...,"Mid-Professional, Professional Individual Cont...",NaN,NaN,B,True,False
...,...,...,...,...,...,...,...,...,...
368,DSI v1.1 Interpretation Report,https://www.shl.com/solutions/products/product...,DSI v1.1 Interpretation Report,NaN,"Portuguese (Brazil), Dutch, Finnish, French, L...",NaN,P,True,False
378,Enterprise Leadership Report 1.0,https://www.shl.com/solutions/products/product...,Assess and benchmark your leaders against ente...,"Director, Executive, Manager, Mid-Professional",NaN,NaN,P,True,False
379,Enterprise Leadership Report 2.0,https://www.shl.com/solutions/products/product...,Assess and benchmark your leaders against ente...,"Director, Executive, Manager, Mid-Professional",NaN,NaN,P,True,False
387,Executive Scenarios,https://www.shl.com/solutions/products/product...,Executive Scenarios is a unique test of Manage...,"Director, Executive, Manager","Italian, English (USA), English International,...",NaN,B,True,False


In [51]:
# Create a copy of the dataframe to avoid SettingWithCopyWarning
df_clean = df.copy()

# Fill NaN values with "default" for object columns
object_columns = df_clean.select_dtypes(include='object').columns
for col in object_columns:
    df_clean[col] = df_clean[col].fillna("default")

# Fill NaN values in duration with the median value
df_clean['duration'] = df_clean['duration'].fillna(df_clean['duration'].median())

# Display the first few rows to verify changes
print("Number of null values after filling:")
print(df_clean.isnull().sum())
print("\nSample of cleaned data:")
df_clean.head()

Number of null values after filling:
name              0
url               0
description       0
job_levels        0
languages         0
duration          0
test_type         0
remote_testing    0
adaptive_irt      0
dtype: int64

Sample of cleaned data:


,name,url,description,job_levels,languages,duration,test_type,remote_testing,adaptive_irt
0,Account Manager Solution,https://www.shl.com/solutions/products/product...,The Account Manager solution is an assessment ...,Mid-Professional,English (USA),49.0,"C,P,A,B",True,True
1,Administrative Professional - Short Form,https://www.shl.com/solutions/products/product...,The Administrative Professional solution is fo...,Entry-Level,English (USA),36.0,"A,K,P",True,True
2,Agency Manager Solution,https://www.shl.com/solutions/products/product...,The Agency Manager solution is for mid-level s...,"Front Line Manager, Manager, Supervisor",English (USA),51.0,"A,B,P,S",True,True
3,Apprentice + 8.0 Job Focused Assessment,https://www.shl.com/solutions/products/product...,The Apprentice + 8.0 Job-Focused Assessment is...,"General Population, Graduate, Entry-Level","English International, German",30.0,"B,P",True,False
4,Apprentice 8.0 Job Focused Assessment,https://www.shl.com/solutions/products/product...,The Apprentice 8.0 Job-Focused Assessment is a...,"Entry-Level, General Population, Graduate","English International, German, French",20.0,"B,P",True,False


In [52]:
# Check for null values in each column
null_values = df_clean.isnull().sum()
print("Null values per column:")
print(null_values)
print("\n")

# Check data types
print("Data types:")
print(df_clean.dtypes)
print("\n")

# Check for consistency issues
print("Column value counts:")
print(f"remote_testing: {df_clean['remote_testing'].value_counts()}")
print(f"adaptive_irt: {df_clean['adaptive_irt'].value_counts()}")
print("\n")

# Check test_type format consistency
print("Sample of test_type values:")
print(df_clean['test_type'].head(10))
print("\n")

# Check duration statistics
print("Duration statistics:")
print(df_clean['duration'].describe())
print("\n")

# Check if any required columns are missing
expected_cols = ['name', 'url', 'description', 'job_levels', 'languages', 
                 'duration', 'test_type', 'remote_testing', 'adaptive_irt']
missing_cols = [col for col in expected_cols if col not in df_clean.columns]
if missing_cols:
    print(f"Missing columns: {missing_cols}")
else:
    print("All expected columns are present")

Null values per column:
name              0
url               0
description       0
job_levels        0
languages         0
duration          0
test_type         0
remote_testing    0
adaptive_irt      0
dtype: int64


Data types:
name               object
url                object
description        object
job_levels         object
languages          object
duration          float64
test_type          object
remote_testing       bool
adaptive_irt         bool
dtype: object


Column value counts:
remote_testing: remote_testing
True    398
Name: count, dtype: int64
adaptive_irt: adaptive_irt
False    351
True      47
Name: count, dtype: int64


Sample of test_type values:
0      C,P,A,B
1        A,K,P
2      A,B,P,S
3          B,P
4          B,P
5      A,B,K,P
6        A,B,P
7      A,B,P,S
8      B,P,S,A
9    P,S,K,B,A
Name: test_type, dtype: object


Duration statistics:
count    398.000000
mean      14.090452
std       10.082624
min        0.000000
25%        9.000000
50%       11.000

In [53]:
# Save the cleaned dataframe to a new CSV file
df_clean.to_csv('../assessment_details_cleaned.csv', index=False)
print("Cleaned data saved to '../assessment_details_cleaned.csv'")

Cleaned data saved to '../assessment_details_cleaned.csv'


In [54]:
df = pd.read_csv('../assessment_details_cleaned.csv')

In [55]:
df.head()

,name,url,description,job_levels,languages,duration,test_type,remote_testing,adaptive_irt
0,Account Manager Solution,https://www.shl.com/solutions/products/product...,The Account Manager solution is an assessment ...,Mid-Professional,English (USA),49.0,"C,P,A,B",True,True
1,Administrative Professional - Short Form,https://www.shl.com/solutions/products/product...,The Administrative Professional solution is fo...,Entry-Level,English (USA),36.0,"A,K,P",True,True
2,Agency Manager Solution,https://www.shl.com/solutions/products/product...,The Agency Manager solution is for mid-level s...,"Front Line Manager, Manager, Supervisor",English (USA),51.0,"A,B,P,S",True,True
3,Apprentice + 8.0 Job Focused Assessment,https://www.shl.com/solutions/products/product...,The Apprentice + 8.0 Job-Focused Assessment is...,"General Population, Graduate, Entry-Level","English International, German",30.0,"B,P",True,False
4,Apprentice 8.0 Job Focused Assessment,https://www.shl.com/solutions/products/product...,The Apprentice 8.0 Job-Focused Assessment is a...,"Entry-Level, General Population, Graduate","English International, German, French",20.0,"B,P",True,False


In [56]:
def clean_list_field(field):
    """Clean list fields that might be string representations of lists or comma-separated strings"""
    if pd.isna(field) or field == '':
        return []
        
    # If it's already a list, return it
    if isinstance(field, list):
        return field
        
    # If it looks like a string representation of a list: ['item1', 'item2']
    if isinstance(field, str) and field.startswith('[') and field.endswith(']'):
        try:
            # Try to eval it safely to convert string representation to actual list
            # Remove quotes for safer eval
            clean_str = field.replace("'", '"')
            result = eval(clean_str)
            if isinstance(result, list):
                return result
        except:
            pass
    
    # Otherwise, treat as comma-separated string
    if isinstance(field, str):
        return [item.strip() for item in field.split(',') if item.strip()]
    
    return []

In [57]:
for col in ['job_levels', 'languages', 'test_type']:
        if col in df.columns:
            df[col] = df[col].apply(clean_list_field)

In [127]:
df_copy = df.copy()

In [136]:
# For job_levels - extract all unique values from the list column
job_levels_unique = set()
for job_levels_list in df_copy['job_levels']:
    if isinstance(job_levels_list, list):
        for level in job_levels_list:
            job_levels_unique.add(level)

# For languages - extract all unique values from the list column
languages_unique = set()
for languages_list in df_copy['languages']:
    if isinstance(languages_list, list):
        for language in languages_list:
            languages_unique.add(language)

# For test_type - extract all unique values from the list column
test_types_unique = set()
for test_types_list in df_copy['test_type']:
    if isinstance(test_types_list, list):
        for test_type in test_types_list:
            test_types_unique.add(test_type)

# Convert sets to sorted lists for better readability
job_levels_unique = sorted(list(job_levels_unique))
languages_unique = sorted(list(languages_unique))
test_types_unique = sorted(list(test_types_unique))

# Display the unique values
print("Unique Job Levels:")
print(job_levels_unique)
print("\nUnique Languages:")
print(languages_unique)
print("\nUnique Test Types:")
print(test_types_unique)

Unique Job Levels:
['Director', 'Entry-Level', 'Executive', 'Front Line Manager', 'General Population', 'Graduate', 'Manager', 'Mid-Professional', 'Professional Individual Contributor', 'Supervisor', 'default']

Unique Languages:
['Arabic', 'Chinese Simplified', 'Chinese Traditional', 'Czech', 'Danish', 'Dutch', 'English (Australia)', 'English (Canada)', 'English (South Africa)', 'English (USA)', 'English International', 'Estonian', 'Finnish', 'Flemish', 'French', 'French (Belgium)', 'French (Canada)', 'German', 'Greek', 'Hungarian', 'Icelandic', 'Indonesian', 'Italian', 'Japanese', 'Korean', 'Latin American Spanish', 'Latvian', 'Lithuanian', 'Malay', 'Norwegian', 'Polish', 'Portuguese', 'Portuguese (Brazil)', 'Romanian', 'Russian', 'Serbian', 'Slovak', 'Spanish', 'Swedish', 'Thai', 'Turkish', 'Vietnamese', 'default']

Unique Test Types:
['A', 'B', 'C', 'D', 'E', 'K', 'P', 'S']


In [128]:
# A = Ability & Aptitude
# B = biodata & situational judgment
# C = competencies
# D = development and 360
# E = Aessessment Exercises
# K = knowledge & skills
# P = personality & behavior 
# s = simulation
# in the job type column we have the following values:
# we have to check if the values are in the list above and if not we have to remove them
# 

In [ ]:
# from langchain_core.documents import Document

# def prepare_documents(df):
#     documents = []
#     for _, row in df.iterrows():
#         # Handle lists properly - they're already processed by the clean_list_field function
#         job_levels = row['job_levels'] if isinstance(row['job_levels'], list) else []
#         languages = row['languages'] if isinstance(row['languages'], list) else []
#         test_types = row['test_type'] if isinstance(row['test_type'], list) else []
        
#         # Create a comprehensive page content
#         job_levels_str = ' and '.join(job_levels) if job_levels else 'various job levels'
#         languages_str = ' and '.join(languages) if languages else 'multiple languages'
#         test_types_str = ', '.join(test_types) if test_types else 'various assessments'
        
#         # Create a dictionary to map test type codes to full descriptions
#         test_type_mapping = {
#             'A': 'Ability & Aptitude',
#             'B': 'Biodata & Situational Judgment',
#             'C': 'Competencies',
#             'D': 'Development and 360',
#             'E': 'Assessment Exercises',
#             'K': 'Knowledge & Skills',
#             'P': 'Personality & Behavior',
#             'S': 'Simulation'
#         }
        
#         # Create formatted test type descriptions
#         test_type_descriptions = []
#         for test_type in test_types:
#             # Standardize the test type (uppercase) and get the description if available
#             test_type = test_type.upper() if isinstance(test_type, str) else test_type
#             description = test_type_mapping.get(test_type, f"Unknown ({test_type})")
#             test_type_descriptions.append(f"{test_type}: {description}")
        
#         test_types_detailed = ", ".join(test_type_descriptions) if test_type_descriptions else "No test types specified"
        
#         # Create the page content with rich information - without unnecessary newlines
#         page_content = (
#             f"{row['name']}: A {job_levels_str} position in {languages_str}. "
#             f"Job Description: {row['description']} "
#             f"Test Types: {test_types_str} "
#             f"Detailed Test Types: {test_types_detailed} "
#             f"Duration: {row['duration']} minutes "
#             f"Remote Testing: {'Available' if row['remote_testing'] else 'Not available'} "
#             f"Adaptive Testing: {'Yes' if row['adaptive_irt'] else 'No'}"
#         )
        
#         # Create document with rich metadata
#         document = Document(
#             page_content=page_content,
#             metadata={
#                 "name": row['name'],
#                 "url": row['url'],
#                 "job_levels": job_levels,
#                 "languages": languages,
#                 "duration": row['duration'],
#                 "test_types": test_types,
#                 "remote_testing": row['remote_testing'],
#                 "adaptive_irt": row['adaptive_irt']
#             }
#         )
        
#         documents.append(document)
    
#     return documents


In [138]:
from langchain_core.documents import Document

def prepare_documents(df):
    documents = []
    
    # Create mappings of all possible values for each category
    test_type_mapping = {
        'A': 'Ability & Aptitude',
        'B': 'Biodata & Situational Judgment',
        'C': 'Competencies',
        'D': 'Development and 360',
        'E': 'Assessment Exercises',
        'K': 'Knowledge & Skills',
        'P': 'Personality & Behavior',
        'S': 'Simulation'
    }
    
    # Get all unique values from the dataframe first (for job levels and languages)
    all_job_levels = set()
    all_languages = set()
    
    for _, row in df.iterrows():
        job_levels = row['job_levels'] if isinstance(row['job_levels'], list) else []
        languages = row['languages'] if isinstance(row['languages'], list) else []
        
        for level in job_levels:
            all_job_levels.add(level.lower())
        for lang in languages:
            all_languages.add(lang.lower())
    
    # Now process each row with all possible values in mind
    for _, row in df.iterrows():
        # Handle lists properly - they're already processed by the clean_list_field function
        job_levels = row['job_levels'] if isinstance(row['job_levels'], list) else []
        languages = row['languages'] if isinstance(row['languages'], list) else []
        test_types = row['test_type'] if isinstance(row['test_type'], list) else []
        
        # Create same page content as before
        job_levels_str = ' and '.join(job_levels) if job_levels else 'various job levels'
        languages_str = ' and '.join(languages) if languages else 'multiple languages'
        test_types_str = ', '.join(test_types) if test_types else 'various assessments'
        
        # Create formatted test type descriptions
        test_type_descriptions = []
        test_type_categories = []
        
        for test_type in test_types:
            test_type = test_type.upper() if isinstance(test_type, str) else test_type
            description = test_type_mapping.get(test_type, f"Unknown ({test_type})")
            test_type_descriptions.append(f"{test_type}: {description}")
            
            if test_type in test_type_mapping:
                test_type_categories.append(test_type_mapping[test_type])
        
        test_types_detailed = ", ".join(test_type_descriptions) if test_type_descriptions else "No test types specified"
        
        # Create the page content
        page_content = (
            f"{row['name']}: A {job_levels_str} position in {languages_str}. "
            f"Job Description: {row['description']} "
            f"Test Types: {test_types_str} "
            f"Detailed Test Types: {test_types_detailed} "
            f"Duration: {row['duration']} minutes "
            f"Remote Testing: {'Available' if row['remote_testing'] else 'Not available'} "
            f"Adaptive Testing: {'Yes' if row['adaptive_irt'] else 'No'}"
        )
        
        # Start with basic metadata using simple types
        metadata = {
            "name": str(row['name']),
            "url": str(row['url']),
            "description": str(row['description']),
            "duration": float(row['duration']) if isinstance(row['duration'], (int, float)) or 
                      (isinstance(row['duration'], str) and row['duration'].replace('.', '', 1).isdigit()) else 0.0,
            "remote_testing": bool(row['remote_testing']),
            "adaptive_irt": bool(row['adaptive_irt']),
            "search_keywords": " ".join([str(row['name']), str(row['description']), 
                                        *[str(level) for level in job_levels],
                                        *[str(lang) for lang in languages], 
                                        *[str(tt) for tt in test_types]]).lower(),
        }
        
        # Add duration range as string
        metadata["duration_range"] = get_duration_range(row['duration'])
        
        # Add job level boolean flags
        for level in all_job_levels:
            metadata[f"job_level_{level.replace(' ', '_').replace('-', '_')}"] = any(jl.lower() == level for jl in job_levels) 
        
        # Add language boolean flags
        for lang in all_languages:
            metadata[f"language_{lang.replace(' ', '_').replace('-', '_').replace('#', 'sharp').replace('+', 'plus')}"] = any(l.lower() == lang for l in languages)
        
        # Add test type boolean flags - both for codes and categories
        for code in test_type_mapping:
            metadata[f"test_type_{code}"] = code in [tt.upper() if isinstance(tt, str) else tt for tt in test_types]
        
        # Add category flags
        metadata["contains_cognitive"] = any(cat in ["Ability & Aptitude", "Knowledge & Skills"] for cat in test_type_categories)
        metadata["contains_personality"] = any(cat in ["Personality & Behavior"] for cat in test_type_categories)
        metadata["contains_technical"] = any(cat in ["Knowledge & Skills", "Simulation"] for cat in test_type_categories)
        metadata["contains_soft_skill"] = any(cat in ["Competencies", "Biodata & Situational Judgment", "Personality & Behavior"] for cat in test_type_categories)
        
        # Add duration-based flags for faster filtering
        metadata["duration_under_30"] = metadata["duration"] <= 30
        metadata["duration_under_45"] = metadata["duration"] <= 45
        metadata["duration_under_60"] = metadata["duration"] <= 60
        
        # Create document with flat metadata structure
        document = Document(
            page_content=page_content,
            metadata=metadata
        )
        
        documents.append(document)
    
    return documents

def get_duration_range(duration):
    """Categorize duration into ranges for easier filtering"""
    try:
        duration = float(duration)
        if duration <= 15:
            return "very_short"
        elif duration <= 30:
            return "short"
        elif duration <= 45:
            return "medium"
        elif duration <= 60:
            return "standard"
        else:
            return "long"
    except (ValueError, TypeError):
        return "unknown"

In [139]:
prepare_data = prepare_documents(df_copy)


In [141]:
prepare_data[0].metadata

{'name': 'Account Manager Solution',
 'url': 'https://www.shl.com/solutions/products/product-catalog/view/account-manager-solution/',
 'description': 'The Account Manager solution is an assessment used for job candidates applying to mid-level leadership positions that tend to manage the day-to-day operations and activities of client accounts. Sample tasks for these jobs include, but are not limited to: communicating with clients about project status, developing and maintaining project plans, coordinating internally with appropriate project personnel, and ensuring client expectations are being met. Potential job titles that use this solution are: Account Executive, Account Manager, and Senior Account Manager. There are multiple configurations of this solution available.',
 'duration': 49.0,
 'remote_testing': True,
 'adaptive_irt': True,
 'search_keywords': 'account manager solution the account manager solution is an assessment used for job candidates applying to mid-level leadership po

In [ ]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# Create a vector store with optimized documents
vector_store = Chroma.from_documents(
    documents=prepare_data,  # Use the optimized documents
    embedding=embeddings,
    persist_directory="../shl_optimized_vector_db"  # Use a different directory for optimized data
)

# Perform a similarity search to verify


Search Results for: Find safety assessments for entry-level warehouse positions
--------------------------------------------------
Result 1:
Content: Workplace Safety Solution: A Entry-Level position in English (USA) and English (Australia). Job Description: The Workplace Safety Solution is designed for entry-level positions to measure the competen...
Name: Workplace Safety Solution
Test types: None
Duration: 21.0 minutes


Result 2:
Content: Workplace Safety - Individual 7.0 Solution: A Entry-Level position in German and English International and English (USA) and Italian and French and Dutch and French (Canada) and Latin American Spanish...
Name: Workplace Safety - Individual 7.0 Solution
Test types: None
Duration: 16.0 minutes


Result 3:
Content: Workplace Safety - Individual 7.1 (Americas): A Entry-Level position in Portuguese (Brazil) and English (USA) and French (Canada) and Latin American Spanish. Job Description: Our Workplace Safety - In...
Name: Workplace Safety - Individual

In [154]:
import re
from typing import Dict, Any

    # Extract key metadata filters from query
    # This could be done with regex patterns or NLP techniques
#     from langchain_core.documents import Document

# def prepare_documents(df):
#     documents = []
    
#     # Create mappings of all possible values for each category
#     test_type_mapping = {
#         'A': 'Ability & Aptitude',
#         'B': 'Biodata & Situational Judgment',
#         'C': 'Competencies',
#         'D': 'Development and 360',
#         'E': 'Assessment Exercises',
#         'K': 'Knowledge & Skills',
#         'P': 'Personality & Behavior',
#         'S': 'Simulation'
#     }
    
#     # Get all unique values from the dataframe first (for job levels and languages)
#     all_job_levels = set()
#     all_languages = set()
    
#     for _, row in df.iterrows():
#         job_levels = row['job_levels'] if isinstance(row['job_levels'], list) else []
#         languages = row['languages'] if isinstance(row['languages'], list) else []
        
#         for level in job_levels:
#             all_job_levels.add(level.lower())
#         for lang in languages:
#             all_languages.add(lang.lower())
    
#     # Now process each row with all possible values in mind
#     for _, row in df.iterrows():
#         # Handle lists properly - they're already processed by the clean_list_field function
#         job_levels = row['job_levels'] if isinstance(row['job_levels'], list) else []
#         languages = row['languages'] if isinstance(row['languages'], list) else []
#         test_types = row['test_type'] if isinstance(row['test_type'], list) else []
        
#         # Create same page content as before
#         job_levels_str = ' and '.join(job_levels) if job_levels else 'various job levels'
#         languages_str = ' and '.join(languages) if languages else 'multiple languages'
#         test_types_str = ', '.join(test_types) if test_types else 'various assessments'
        
#         # Create formatted test type descriptions
#         test_type_descriptions = []
#         test_type_categories = []
        
#         for test_type in test_types:
#             test_type = test_type.upper() if isinstance(test_type, str) else test_type
#             description = test_type_mapping.get(test_type, f"Unknown ({test_type})")
#             test_type_descriptions.append(f"{test_type}: {description}")
            
#             if test_type in test_type_mapping:
#                 test_type_categories.append(test_type_mapping[test_type])
        
#         test_types_detailed = ", ".join(test_type_descriptions) if test_type_descriptions else "No test types specified"
        
#         # Create the page content
#         page_content = (
#             f"{row['name']}: A {job_levels_str} position in {languages_str}. "
#             f"Job Description: {row['description']} "
#             f"Test Types: {test_types_str} "
#             f"Detailed Test Types: {test_types_detailed} "
#             f"Duration: {row['duration']} minutes "
#             f"Remote Testing: {'Available' if row['remote_testing'] else 'Not available'} "
#             f"Adaptive Testing: {'Yes' if row['adaptive_irt'] else 'No'}"
#         )
        
#         # Start with basic metadata using simple types
#         metadata = {
#             "name": str(row['name']),
#             "url": str(row['url']),
#             "description": str(row['description']),
#             "duration": float(row['duration']) if isinstance(row['duration'], (int, float)) or 
#                       (isinstance(row['duration'], str) and row['duration'].replace('.', '', 1).isdigit()) else 0.0,
#             "remote_testing": bool(row['remote_testing']),
#             "adaptive_irt": bool(row['adaptive_irt']),
#             "search_keywords": " ".join([str(row['name']), str(row['description']), 
#                                         *[str(level) for level in job_levels],
#                                         *[str(lang) for lang in languages], 
#                                         *[str(tt) for tt in test_types]]).lower(),
#         }
        
#         # Add duration range as string
#         metadata["duration_range"] = get_duration_range(row['duration'])
        
#         # Add job level boolean flags
#         for level in all_job_levels:
#             metadata[f"job_level_{level.replace(' ', '_').replace('-', '_')}"] = any(jl.lower() == level for jl in job_levels) 
        
#         # Add language boolean flags
#         for lang in all_languages:
#             metadata[f"language_{lang.replace(' ', '_').replace('-', '_').replace('#', 'sharp').replace('+', 'plus')}"] = any(l.lower() == lang for l in languages)
        
#         # Add test type boolean flags - both for codes and categories
#         for code in test_type_mapping:
#             metadata[f"test_type_{code}"] = code in [tt.upper() if isinstance(tt, str) else tt for tt in test_types]
        
#         # Add category flags
#         metadata["contains_cognitive"] = any(cat in ["Ability & Aptitude", "Knowledge & Skills"] for cat in test_type_categories)
#         metadata["contains_personality"] = any(cat in ["Personality & Behavior"] for cat in test_type_categories)
#         metadata["contains_technical"] = any(cat in ["Knowledge & Skills", "Simulation"] for cat in test_type_categories)
#         metadata["contains_soft_skill"] = any(cat in ["Competencies", "Biodata & Situational Judgment", "Personality & Behavior"] for cat in test_type_categories)
        
#         # Add duration-based flags for faster filtering
#         metadata["duration_under_30"] = metadata["duration"] <= 30
#         metadata["duration_under_45"] = metadata["duration"] <= 45
#         metadata["duration_under_60"] = metadata["duration"] <= 60
        
#         # Create document with flat metadata structure
#         document = Document(
#             page_content=page_content,
#             metadata=metadata
#         )
        
#         documents.append(document)
    
#     return documents

# def get_duration_range(duration):
#     """Categorize duration into ranges for easier filtering"""
#     try:
#         duration = float(duration)
#         if duration <= 15:
#             return "very_short"
#         elif duration <= 30:
#             return "short"
#         elif duration <= 45:
#             return "medium"
#         elif duration <= 60:
#             return "standard"
#         else:
#             return "long"
#     except (ValueError, TypeError):
#         return "unknown"
def extract_filters_from_query(query: str) -> Dict[str, Any]:
    """Extract metadata filters from a natural language query."""
    
    filter_conditions = []
    
    # Job level detection
    job_levels = [
        "analyst", "director", "entry-level", "executive", "front line manager",
        "general population", "graduate", "manager", "mid-professional", 
        "professional individual contributor", "supervisor"
    ]
    
    for level in job_levels:
        if re.search(r'\b' + re.escape(level) + r'\b', query.lower()):
            filter_conditions.append(
                {"job_level_{0}".format(level.replace(' ', '_').replace('-', '_')): True}
            )
    
    # Test type categories detection
    if re.search(r'\b(cognitive|ability|aptitude)\b', query.lower()):
        filter_conditions.append({"contains_cognitive": True})
    
    if re.search(r'\b(personality|behavior|behaviour)\b', query.lower()):
        filter_conditions.append({"contains_personality": True})
        
    if re.search(r'\b(technical|knowledge|skill)\b', query.lower()):
        filter_conditions.append({"contains_technical": True})
        
    if re.search(r'\b(soft skill|competenc|situational|judgment)\b', query.lower()):
        filter_conditions.append({"contains_soft_skill": True})
    
    # Duration detection
    duration_match = re.search(r'(\d+)\s*(min|mins|minutes)', query.lower())
    if duration_match:
        minutes = int(duration_match.group(1))
        if minutes <= 15:
            filter_conditions.append({"duration_range": "very_short"})
        elif minutes <= 30:
            filter_conditions.append({"duration_range": "short"})
        elif minutes <= 45:
            filter_conditions.append({"duration_under_45": True})
        elif minutes <= 60:
            filter_conditions.append({"duration_under_60": True})
        
    # Language detection
    languages = [
        "arabic", "chinese simplified", "chinese traditional", "czech", "danish",
        "dutch", "english", "estonian", "finnish", "flemish", "french", "german",
        "greek", "hungarian", "icelandic", "indonesian", "italian", "japanese",
        "korean", "latvian", "lithuanian", "malay", "norwegian", "polish",
        "portuguese", "romanian", "russian", "serbian", "slovak", "spanish",
        "swedish", "thai", "turkish", "vietnamese"
    ]
    
    for lang in languages:
        if re.search(r'\b' + re.escape(lang) + r'\b', query.lower()):
            clean_lang = lang.replace(' ', '_').replace('-', '_')
            filter_conditions.append({f"language_{clean_lang}": True})
    
    # Remote testing and adaptive features
    if "remote" in query.lower():
        filter_conditions.append({"remote_testing": True})
        
    if "adaptive" in query.lower():
        filter_conditions.append({"adaptive_irt": True})
    
    # Return proper ChromaDB filter structure
    if not filter_conditions:
        return None  # No filters found
    elif len(filter_conditions) == 1:
        return filter_conditions[0]  # Single filter
    else:
        return {"$or": filter_conditions}  # Multiple filters combined with AND
    # For this example query: "analyst", "cognitive", "personality", "45 mins"
    # filters = {
    #     "job_level_analyst": True,
    #     "contains_cognitive": True, 
    #     "contains_personality": True,
    #     "duration_under_45": True
    # }
    
    # Perform filtered vector search


In [ ]:
query = "I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes."

results = vector_store.similarity_search_with_score(
    query=query,
    k=5,  
    filter=extract_filters_from_query(query),  # Use the extracted filters
)

results

[(Document(id='ebc252ed-fcd3-427d-9922-fb64bf29e671', metadata={'language_swedish': False, 'language_malay': False, 'duration_range': 'short', 'test_type_E': False, 'language_norwegian': False, 'job_level_graduate': False, 'language_russian': False, 'language_italian': False, 'language_danish': False, 'language_turkish': False, 'job_level_default': False, 'remote_testing': True, 'language_serbian': False, 'duration_under_60': True, 'language_german': False, 'language_latvian': False, 'language_indonesian': False, 'job_level_supervisor': False, 'language_dutch': False, 'test_type_S': False, 'name': 'Java 2 Platform Enterprise Edition 1.4 Fundamental', 'test_type_C': False, 'language_estonian': False, 'language_english_(canada)': False, 'test_type_K': True, 'language_english_(usa)': True, 'job_level_executive': False, 'language_icelandic': False, 'language_vietnamese': False, 'language_french_(belgium)': False, 'contains_technical': True, 'language_english_(australia)': False, 'language_

In [160]:
print("Search Results for:", query)
print("-" * 50)
for i, (result, score) in enumerate(results):
    print(f"Result {i+1} (Similarity Score: {score:.4f}):")
    print(f"Content: {result.page_content[:200]}...") 
    print(f"URL: {result.metadata.get('url')}")
    print(f"Name: {result.metadata.get('name')}")
    print(f"Test types: {', '.join([t for t in result.metadata if t.startswith('test_type_') and result.metadata[t]])}")
    print(f"Duration: {result.metadata.get('duration')} minutes")
    print(f"Contains Cognitive: {result.metadata.get('contains_cognitive', False)}")
    print(f"Contains Personality: {result.metadata.get('contains_personality', False)}")
    print("\n")

Search Results for: I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.
--------------------------------------------------
Result 1 (Similarity Score: 0.5745):
Content: Java 2 Platform Enterprise Edition 1.4 Fundamental: A Entry-Level and Mid-Professional and Professional Individual Contributor position in English (USA). Job Description: The Java 2 Platform Enterpris...
URL: https://www.shl.com/solutions/products/product-catalog/view/java-2-platform-enterprise-edition-1-4-fundamental/
Name: Java 2 Platform Enterprise Edition 1.4 Fundamental
Test types: test_type_K
Duration: 30.0 minutes
Contains Cognitive: True
Contains Personality: False


Result 2 (Similarity Score: 0.5767):
Content: Java 8 (New): A Mid-Professional and Professional Individual Contributor position in English (USA). Job Description: Multi-choice test that measures the knowledge of Java class design, exceptions, g

In [163]:
# if any url of website given as search query then we have to parse the content of that website using langchain webbase and using gemini ai we have to make query and create filter and do search

import re
from urllib.parse import urlparse
from typing import Tuple, Optional, Dict, Any

from langchain_community.document_loaders import WebBaseLoader
from langchain_google_genai import GoogleGenerativeAI

def is_valid_url(url: str) -> bool:
    """Check if the provided string is a valid URL."""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except:
        return False

def extract_url_from_query(query: str) -> Tuple[str, Optional[str]]:
    """Extract URL from a search query if present."""
    # Simple regex to find URLs in text
    url_pattern = r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    urls = re.findall(url_pattern, query)
    
    if urls:
        url = urls[0]
        # Remove the URL from the query
        clean_query = query.replace(url, "").strip()
        return clean_query, url
    return query, None

def process_webpage_content(url: str) -> str:
    """Fetch and process webpage content."""
    try:
        # Use WebBaseLoader to fetch the page content
        loader = WebBaseLoader(url)
        docs = loader.load()
        
        # Combine all document contents
        content = " ".join([doc.page_content for doc in docs])
        
        # Truncate if too long (Gemini has context limits)
        max_length = 10000  # Adjust based on model requirements
        if len(content) > max_length:
            content = content[:max_length] + "..."
            
        return content
    except Exception as e:
        return f"Error loading webpage: {str(e)}"

def analyze_webpage_with_gemini(content: str) -> Tuple[str, Dict[str, Any]]:
    """Use Gemini to analyze webpage content and extract search criteria."""
    try:
        # Initialize Gemini
        gemini = GoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25", temperature=0)
        
        # Prompt asking Gemini to analyze the content
        prompt = f"""
        Analyze the following webpage content and extract key information to help find relevant assessments:
        
        {content}
        
        Based on this content, please:
        1. Identify the job role or position mentioned
        2. Extract any skills or competencies required
        3. Determine job level (entry, manager, executive, etc.)
        4. Identify technical skills needed
        5. Determine if personality or cognitive assessments would be relevant
        6. Extract any timing constraints
        
        Format your response as a structured JSON with these fields: 
        job_role, skills, job_level, technical_skills, assessment_types, duration
        """
        
        # Get Gemini's analysis
        response = gemini.invoke(prompt)
        
        # Process the response to extract a search query and filters
        # This is a simplification - you might need to parse JSON from the response
        generated_query = f"Assessment for {response.split('job_role')[1].split(',')[0]} position"
        
        # Extract filters from the generated query using our existing function
        filters = extract_filters_from_query(response)
        
        return generated_query, filters
    except Exception as e:
        return f"Error analyzing content: {str(e)}", None

def search_with_url(query: str, vector_store) -> list:
    """Process a search query that may contain a URL."""
    # Extract URL if present
    clean_query, url = extract_url_from_query(query)
    
    if url and is_valid_url(url):
        print(f"Found URL in query: {url}")
        
        # Fetch and process the webpage content
        content = process_webpage_content(url)
        
        # If content was successfully retrieved
        if not content.startswith("Error"):
            # Analyze the content with Gemini
            generated_query, url_based_filters = analyze_webpage_with_gemini(content)
            
            # If analysis was successful
            if not isinstance(generated_query, str) or not generated_query.startswith("Error"):
                print(f"Generated search query: {generated_query}")
                
                # Combine with any filters from the original query
                query_filters = extract_filters_from_query(clean_query)
                combined_filters = merge_filters(url_based_filters, query_filters)
                
                # Perform the search
                return vector_store.similarity_search_with_score(
                    query=generated_query,
                    k=5,
                    filter=combined_filters
                )
    
    # Fallback to regular search if URL processing fails or no URL present
    return vector_store.similarity_search_with_score(
        query=query,
        k=5,
        filter=extract_filters_from_query(query)
    )

def merge_filters(filters1, filters2):
    """Merge two filter dictionaries, combining them with OR logic."""
    if not filters1:
        return filters2
    if not filters2:
        return filters1
    
    # Combine filters with OR logic
    return {"$or": [filters1, filters2]}

# Example usage
# query = "I found this job posting https://example.com/job-posting. Can you recommend an assessment?"
# results = search_with_url(query, vector_store)

In [164]:
# Use the new search function that can handle URLs
query = "Here is a https://www.linkedin.com/jobs/view/research-engineer-ai-at-shl-4194768899/?originalSubdomain=in , can you recommend some assessment that can help me screen applications. Time limit is less than 30 minutes."
# Or a query with URL: "I found this job posting https://example.com/java-developer-job. What assessment should I use?"

results = search_with_url(query, vector_store)

# Display results as before
print("Search Results for:", query)
print("-" * 50)
for i, (result, score) in enumerate(results):
    print(f"Result {i+1} (Similarity Score: {score:.4f}):")
    print(f"Content: {result.page_content[:200]}...") 
    print(f"URL: {result.metadata.get('url')}")
    print(f"Name: {result.metadata.get('name')}")
    print(f"Test types: {', '.join([t for t in result.metadata if t.startswith('test_type_') and result.metadata[t]])}")
    print(f"Duration: {result.metadata.get('duration')} minutes")
    print(f"Contains Cognitive: {result.metadata.get('contains_cognitive', False)}")
    print(f"Contains Personality: {result.metadata.get('contains_personality', False)}")
    print("\n")

Found URL in query: https://www.linkedin.com
Generated search query: Assessment for ": "Not specified in the content. The page is a general landing/login page for LinkedIn and lists various job *categories* (e.g. position
Search Results for: Here is a https://www.linkedin.com/jobs/view/research-engineer-ai-at-shl-4194768899/?originalSubdomain=in , can you recommend some assessment that can help me screen applications. Time limit is less than 30 minutes.
--------------------------------------------------
Result 1 (Similarity Score: 0.6091):
Content: Verify Interactive G+ Candidate Report: A default position in default. Job Description: Verify Interactive G+ Candidate Report Test Types: A Detailed Test Types: A: Ability & Aptitude Duration: 11.0 m...
URL: https://www.shl.com/solutions/products/product-catalog/view/verify-interactive-g-candidate-report/
Name: Verify Interactive G+ Candidate Report
Test types: test_type_A
Duration: 11.0 minutes
Contains Cognitive: True
Contains Personality

In [165]:
import os
import re
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

In [175]:
def extract_url_from_query(query):
    """Extract URLs from the user query."""
    url_pattern = r'https?://[^\s]+'
    urls = re.findall(url_pattern, query)
    return urls[0] if urls else None

def extract_job_description(url):
    """Extract job description from a job listing webpage."""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # First try to find job description by common class names or IDs
        job_desc_selectors = [
            'div.description', 'div.job-description', '#job-description',
            '.job-details', '.description', '[data-test="job-description"]',
            'section.description', 'div.details', '.details-pane', 
            '.job-desc', '.show-more-less-html'
        ]
        
        for selector in job_desc_selectors:
            job_desc = soup.select_one(selector)
            if job_desc:
                return job_desc.get_text(separator='\n', strip=True)
        
        # If specific selectors fail, use a more generic approach
        # Find sections with job-related terms in them
        job_keywords = ['responsibilities', 'requirements', 'qualifications', 'about the job', 'job summary', 'what you&nbspll do', 'what we&nbspre looking for']
        
        for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'strong']):
            heading_text = heading.get_text().lower()
            if any(keyword in heading_text for keyword in job_keywords):
                # Get the next sibling elements which likely contain the job description
                description = []
                current = heading.find_next_sibling()
                while current and current.name not in ['h1', 'h2', 'h3', 'h4']:
                    if current.get_text(strip=True):
                        description.append(current.get_text(strip=True))
                    current = current.find_next_sibling()
                if description:
                    return '\n'.join(description)
        
        # If all else fails, get the main content area and try to extract job details
        main_content = soup.find('main') or soup.find('article') or soup.find('div', class_='content')
        if main_content:
            text = main_content.get_text(separator='\n', strip=True)
            # Clean up the text
            lines = (line.strip() for line in text.splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            return '\n'.join(chunk for chunk in chunks if chunk)
        
        # Last resort: just get the page title and any text
        title = soup.title.string if soup.title else "Job Listing"
        return f"{title}{soup.get_text( strip=True)[:2000]}"
        
    except Exception as e:
        return f"Error extracting job description: {str(e)}"

def generate_search_query(job_description):
    """Generate a search query based on job description using Gemini."""
    prompt = f"""
    Based on the following job description, create a concise search query to find appropriate
    assessment tests that would help screen candidates for this position.
    
    JOB DESCRIPTION:
    {job_description[:3000]}  # Limit to avoid token limits
    
    Focus on:
    1. Technical skills required
    2. Soft skills mentioned
    3. Any specific assessment requirements
    4. Time constraints for assessments if mentioned
    
    Return ONLY the search query, nothing else.
    """
    
    model = GoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25")
    response = model.invoke(prompt)
    return response.strip()

def search_assessments(query):
    """Search for assessments based on the query."""
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    
    # In a real system, you would load your existing vector store
    vector_store = Chroma(
        persist_directory="../shl_optimized_vector_db",
        embedding_function=embeddings
    )
    
    # Extract any time constraints from the query
    time_pattern = r'(\d+)\s*minutes'
    time_match = re.search(time_pattern, query.lower())
    max_duration = None
    
    if time_match:
        max_duration = int(time_match.group(1))
    
    # Perform the search
    results = vector_store.similarity_search(
        query=query,
        k=5
    )
    
    # If we have time constraints, filter results manually
    if max_duration:
        filtered_results = []
        for doc in results:
            duration = doc.metadata.get('duration')
            if duration and int(duration) <= max_duration:
                filtered_results.append(doc)
        return filtered_results
    
    return results

def process_user_query(user_query):
    """Process user query with URL to extract job description and search for assessments."""
    # Extract URL from query
    url = extract_url_from_query(user_query)
    if not url:
        return "No URL found in your query. Please include a URL to a job listing."
    
    # Extract job description from URL
    job_description = extract_job_description(url)
    if job_description.startswith("Error"):
        return job_description
    
    # Generate search query based on job description
    search_query = generate_search_query(job_description)
    
    # Incorporate any time constraints from the original query
    time_pattern = r'(\d+)\s*minutes'
    time_match = re.search(time_pattern, user_query.lower())
    if time_match:
        max_duration = time_match.group(0)
        if "time" not in search_query.lower() and "minute" not in search_query.lower():
            search_query += f" Assessment duration less than {max_duration}."
    
    try:
        # Search for assessments
        results = search_assessments(search_query)
        
        # Format results
        formatted_results = "\nRecommended Assessments:\n" + "-" * 40 + "\n"
        if not results:
            formatted_results += "No matching assessments found."
        else:
            for i, result in enumerate(results, 1):
                formatted_results += f"Assessment {i}:\n"
                formatted_results += f"Name: {result.metadata.get('name', 'N/A')}\n"
                formatted_results += f"Duration: {result.metadata.get('duration', 'N/A')} minutes\n"
                
                # Format test types
                test_types = [t.replace('test_type_', '') for t in result.metadata 
                              if t.startswith('test_type_') and result.metadata[t]]
                formatted_results += f"Test types: {', '.join(test_types) if test_types else 'N/A'}\n"
                
                formatted_results += f"Description: {result.page_content[:200]}...\n\n"
        
        # Return summary
        summary = f"""
Job Description URL: {url}

Generated Search Query: "{search_query}"

{formatted_results}
        """
        return summary
        
    except Exception as e:
        return f"Error searching for assessments: {str(e)}"

In [176]:
user_query = "Here is a https://www.linkedin.com/jobs/view/research-engineer-ai-at-shl-4194768899/?originalSubdomain=in, can you recommend some assessment that can help me screen applications. Time limit is less than 30 minutes."
    
result = process_user_query(user_query)
print(result)


Job Description URL: https://www.linkedin.com/jobs/view/research-engineer-ai-at-shl-4194768899/?originalSubdomain=in,

Generated Search Query: "AI ML assessment test Python NLP Computer Vision problem solving collaboration Assessment duration less than 30 minutes."


Recommended Assessments:
----------------------------------------
Assessment 1:
Name: AI Skills
Duration: 16.0 minutes
Test types: P
Description: AI Skills: A General Population position in English (USA). Job Description: The AI Skills assessment measures the skills that help candidates successfully leverage AI in their work. Test Types: P Deta...

Assessment 2:
Name: Python (New)
Duration: 11.0 minutes
Test types: K
Description: Python (New): A Mid-Professional and Professional Individual Contributor position in English (USA). Job Description: Multi-choice test that measures the knowledge of Python programming, databases, mod...

Assessment 3:
Name: Verify Interactive G+ Candidate Report
Duration: 11.0 minutes
Test types